<a href="https://colab.research.google.com/github/stefanmzeidler/HI744_Programming_Assignment_1/blob/main/Zeidler_Programming_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs, downloads, and imports

In [1]:
!pip install gensim
!pip install nltk

In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Util Functions

In [3]:
#Provided by Professor He
import nltk
import json
import os
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import json
import sys

def read_txt_files_from_directory(directory_path):
    file_contents = {}
    try:
        for filename in os.listdir(directory_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(directory_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        file_contents[filename] = file.read()
                except Exception as e:
                    print(f"An error occurred while reading {filename}: {e}")
    except Exception as e:
        print(f"An error occurred while accessing the directory: {e}")
        return {}
    return file_contents

def load_from_json(filename):
    try:
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        return data
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

def remove_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_sentence = []
    for token in tokens:
        if token not in stop_words:
            filtered_sentence.append(token)
    return filtered_sentence

def stemming(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

def pre_process(text):
    text_lower = text.lower()
    tokens_no_punctuation = remove_punctuation(text_lower)
    filtered_tokens = remove_stop_words(tokens_no_punctuation)
    stemmed_tokens = stemming(filtered_tokens)
    return stemmed_tokens

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def top_5(vector_matrix, data, column_name):
  sim_matrix = cosine_similarity(vector_matrix)
  top5_indices = np.argpartition(-sim_matrix, range(6), axis=1)[:, 1:6].tolist()
  data[column_name] = top5_indices
  data[column_name] = data[column_name].apply(lambda x: id_list(data,x))

def id_list(data,index_list):
  id_list = []
  for i in index_list:
    id_list.append(data.at[i,'patient_uid'])
  return id_list


# TF-IDF Similarity

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_similarity(data):
  tfidf = TfidfVectorizer()
  vector_matrix = tfidf.fit_transform(data['patient'])
  top_5(vector_matrix, data, 'top5_tfidf')

# Doc2Vec Similarity

In [7]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

def load_dataset(fname,nrows=1000):
  data = safe_read_csv(fname,nrows)
  data['tokens'] = data['patient'].apply(lambda text: pre_process(text))
  print("Data loaded")
  return data

def safe_read_csv(fname,nrows):
  try:
    filepath = os.path.join(PROJ_DIRECTORY,fname)
    return pd.read_csv(filepath_or_buffer=filepath,nrows=nrows)
  except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    return None

#Adapted from https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py
def read_corpus(data, tokens_only = False):
    documents = data['tokens']
    for i, tokens in documents.items():
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

def train_doc2vec(data):
  model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)
  train_corpus = list(read_corpus(data))
  test_corpus = list(read_corpus(data, tokens_only=True))
  model.build_vocab(train_corpus)
  print("Vocab built")
  print("Starting training")
  model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
  fname = get_tmpfile(os.path.join(PROJ_DIRECTORY,"my_doc2vec_model"))
  model.save(fname)
  print("Model saved")
  return model

def doc2vec_vectors(model, data):
  vector_matrix = []
  print("Calcualting Doc2Vec vectors")
  for _ , tokens in data['tokens'].items():
    vector_matrix.append(np.array(model.infer_vector(tokens)))
  print("Vectors calculated")
  return vector_matrix


def load_model(fname):
  return Doc2Vec.load(fname)

def doc2vec_similarity(data):
  doc2vec_model = train_doc2vec(data)
  vector_matrix = doc2vec_vectors(doc2vec_model,data)
  top_5(vector_matrix, data,'top5_doc2vec')







In [8]:
PROJ_DIRECTORY ="/content/drive/MyDrive/HI744_Programming_Assignment_1"
data = load_dataset("PMC-Patients.csv")
doc2vec_similarity(data)
print(data.head())
tfidf_similarity(data)
print(data.head())





Data loaded
Vocab built
Starting training
Model saved
Calcualting Doc2Vec vectors
Vectors calculated
   patient_id patient_uid      PMID                         file_path  \
0           0   7665777-1  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
1           1   7665777-2  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
2           2   7665777-3  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
3           3   7665777-4  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
4           4   7665777-5  33492400  comm/PMC007xxxxxx/PMC7665777.xml   

                                               title  \
0  Early Physical Therapist Interventions for Pat...   
1  Early Physical Therapist Interventions for Pat...   
2  Early Physical Therapist Interventions for Pat...   
3  Early Physical Therapist Interventions for Pat...   
4  Early Physical Therapist Interventions for Pat...   

                                             patient               age gender  \
0  This 60-year-old male was hospitalized 